In [1]:
# GRABIM TEST
%load_ext autoreload
%autoreload 2
import numpy as np

import GRABIM as GRABIM


# Plot (external libraries)
from bokeh.io import output_notebook,show
from bokeh.plotting import figure
from bokeh.models import Legend, LegendItem
from bokeh.models import Arrow, NormalHead
from bokeh.models import ColumnDataSource, LabelSet, Text
output_notebook()

Loading BokehJS ...

In [2]:
# Problem input
f1 = 0.3/(2*np.pi); # [Hz] Start frequency
f2 = 1/(2*np.pi); # [Hz] End frequency
m = 10; # Number of frequency samples
freq = np.array([47.75, 60.13, 72.5, 84.88, 97.26, 109.6, 122, 134.4, 146.8, 159.2])*1e-3;

ZS = [0.713-0.452j, 0.594-0.491j, 0.480-0.500j, 0.377-0.485j, 0.287-0.452j, 0.212-0.409j, 0.151-0.358j, 0.103-0.304j, 0.066-0.248j, 0.038-0.192j]; # [Ohm] Source impedance
ZL = [0.590+0.717j, 0.695+0.744j, 0.769+0.763j, 0.820+0.784j, 0.857+0.809j, 0.884+0.837j, 0.904+0.870j, 0.919+0.906j, 0.932+0.944j, 0.941+0.985j]; # [Ohm] Load impedance

In [3]:
# Candidate topology
code = [ 'LP', 'LS', 'CP', 'CS'];

In [4]:
def Objective_Function(x, *args):
    ZS = args[0]
    ZL = args[1]
    freq = args[2] # Frequency vector
    code = args[3] # Ladder structure
    
    rho_k = GRABIM.LadderCircuitAnalysis.get_Input_Reflection_Coeff(ZS, ZL, code, x, freq);
    max_rho_k = 20*np.log10(max(np.abs(rho_k)));# Get the maximum and convert into dB scale
    
    
    return max_rho_k

In [5]:
from scipy.optimize import differential_evolution
import time

bounds = [];
for i in range(0, len(code)):
    if ((code[i] == 'LS') or (code[i] == 'LP')):
        bounds.append((1e-12, 100))# Bounds for inductance
    elif ((code[i] == 'CS') or (code[i] == 'CP')):
        bounds.append((1e-15, 100))# Bounds for capacitance
bounds = tuple(bounds);

params = (ZS,  ZL, freq, code)

start_time = time.time()
result = differential_evolution(Objective_Function, bounds, args=params, seed=1)
stop_time = time.time()
exec_time = stop_time - start_time;


print("Status:", result['message'])
print("rho_max:", "{:.2f}".format(result['fun']), 'dB')
print("x_best = ", result['x'])
print("Number of iterations:", result["nit"])
print("Execution time: ", "{:.2f}".format(exec_time), 's')


GRABIM.Plot_S21_S11_dB(ZS, ZL, code, result['x'], freq, plotw=1)

Status: Optimization terminated successfully.
rho_max: -6.01 dB
x_best =  [0.87061342 0.71617838 2.37729452 2.38177615]
Number of iterations: 58
Execution time:  6.66 s
